# DDPM原理
DDPM（Denoising Diffusion Probabilistic Models）是Stable Diffusion的基础框架，核心通过前向加噪和反向去噪的马尔可夫链实现生成：

1. 前向加噪过程
   - 核心：从原始潜在表示z0开始，逐步添加高斯噪声，最终变为纯噪声zT~N(0,I)
   - 单步加噪公式：q(zt | zt-1) = N(zt; √(1-βt)·zt-1, βt·I)
     其中βt为噪声调度参数，控制每步噪声强度
   - 直接从z0到zt：zt = √(ᾱt)·z0 + √(1-ᾱt)·ε，ε~N(0,I)
     其中ᾱt = ∏(s=1 to t)(1-βs)为累积缩放因子
   - 噪声调度：Stable Diffusion采用余弦调度，使βt变化更平滑：
     ᾱt = f(t)/f(0), f(t) = cos[(t/T + s)/(1+s)·π/2]^2（s为小偏移量）

2. 反向去噪过程
   - 核心：从纯噪声zT出发，逐步去噪恢复z0，再经VAE解码为图像
   - 去噪分布：pθ(zt-1 | zt) = N(zt-1; μθ(zt,t), Σθ(t))
   - 均值预测：μθ(zt,t,c) = 1/√(1-βt)·[zt - βt/√(1-ᾱt)·εθ(zt,t,c)]
     其中εθ为U-Net预测的噪声，c为文本等条件信息
   - 单步采样：zt-1 = μθ(zt,t,c) + √(βt)·z，z~N(0,I)

3. 训练目标
   - 最小化预测噪声与真实噪声的MSE：
     L = E[t, z0, ε, c][‖ε - εθ(zt,t,c)‖²]
"""

# DDIM作用总结
DDIM（Denoising Diffusion Implicit Models）是Stable Diffusion中用于优化采样的调度器，作用如下：

1. 加速生成过程
   - 替代DDPM的随机采样，采用确定性路径，将采样步数从~1000步压缩至20-50步，大幅提升推理速度

2. 增强可控性
   - 确定性采样：生成路径可复现，避免DDPM的随机性导致的结果波动
   - 多样性调节：通过参数η控制随机性（η=0为纯确定性，η=1接近DDPM）

3. 保持生成质量
   - 尽管步数减少，仍能保持与DDPM接近的图像保真度，尤其适合高分辨率生成

4. 与Stable Diffusion的配合
   - 训练阶段仍用DDPM目标，推理阶段用DDIM加速，平衡训练效果与实用效率

5. 核心公式（确定性采样）
   xt-1 = √(ᾱt-1)·[ (xt - √(1-ᾱt)·ε̂ ) / √(ᾱt) ] + √(1-ᾱt-1)·ε̂
   其中ε̂为模型预测的噪声，通过x0估计推导得出
